In [1]:
## IMPORT LIBRARIES ----------------------------------------------------------------------------------------------------

%matplotlib qt

%reload_ext autoreload
%autoreload 3

import sys                                                                                                              # Import sys to add paths to libraries
import re                                                                                                               # Import re to work with regular expressions
import glob                                                                                                             # Import glob to read files
import matplotlib.pyplot as plt                                                                                         # Import matplotlib.pyplot to plot figures
import tkinter as tk                                                                                                    # Import TK to open folder dialogs to select folders
from tkinter import filedialog                                                                                          # Import filedialog to open folder dialogs to select folders
import numpy                                                                                                            # Import numpy to work with arrays and make calculations
from shutil import rmtree
import random                                                                                                           # Import random to make random choices
from datetime import datetime, timedelta                                                                                # Import time to measure time 
import time                                                                                                             # Import time to measure time
import os                                                                                                               # Import path to work with paths
import pandas                                                                                                           # Import pandas to work with dataframes
import warnings                                                                                                         # Import warnings to ignore warnings
warnings.filterwarnings('ignore')                                                                                       # Ignore warnings

## IMPORT CIRCADIPY ----------------------------------------------------------------------------------------------------

parent_path = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(parent_path)
import chrono_reader as chr                                                                                             # Import chrono_reader to read data
import chrono_plotter as chp                                                                                            # Import chrono_plotter to plot data
import chrono_rhythm as chrt                                                                                             # Import chrono_rithm to make calculations
import chrono_simulation as chs                                                                                         # Import chrono_simulation to simulate data


PyMICE library v. 1.2.1

The library is available under GPL3 license; we ask that reference to our paper
as well as to the library itself is provided in any published research making
use of PyMICE. Please run:

>>> print(pm.__REFERENCING__)

for more information (given that the library is imported as `pm`).




### SELECT THE FILE TO SAVE THE SIMULATIONS

In [2]:
root = tk.Tk()                                                                                                          # Create the root window
root.attributes('-topmost',True)                                                                                        # Keep the window on top of others
root.iconify()                                                                                                          # Hide the root window

main_save_folder = filedialog.askdirectory(title="Select the folder to save the simulated data", parent=root)           # Ask for the folder to save the simulated data
root.destroy()                                                                                                          # Destroy the root window

print(main_save_folder)                                                                                                 # Print the folder to save the simulated data

E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy


In [19]:
def reset_folder(folder):                                                                                               # Function to reset the folder
    for root, dirs, files in os.walk(folder):                                                                           # For each file in the folder
        for f in files:                                                                                                 # For each file
            os.unlink(os.path.join(root, f))                                                                            # Remove the file
        for d in dirs:                                                                                                  # For each folder
            rmtree(os.path.join(root, d))                                                                               # Remove the folder

### RUN AND SAVE THE SIMULATIONS

Each simulation has X cycles (num_cycles), each cycle can be configured with a single activity period. Example: for simulations with a light cycle of 12 hours light and 12 hours dark for 5 days, you have 5 days of this cycle (cycle_days) with a period (activity_period) of 24 hours. 

In [20]:
## CREATE THE FOLDERS --------------------------------------------------------------------------------------------------

remove_file = False                                                                                                     # If the file should be removed

save_folder = main_save_folder + '/data_circadipy/'                                                                     # Define the folder to save the data
result_folder = main_save_folder + '/results_circadipy/'                                                                # Define the folder to save the results

if not os.path.exists(save_folder):
    os.makedirs(save_folder)
else:
    if remove_file:                                                                                                     # If the file should be removed
        reset_folder(save_folder)                                                                                       # Reset the folder
        print("Data folder reseted")                                                                                    # Print that the folder was reseted
    else:                                                                                                               # If the file shouldn't be removed
        print("Data folder already exists")                                                                             # Print that the folder already exists

if not os.path.exists(result_folder):
    os.makedirs(result_folder)
else:
    if remove_file:                                                                                                     # If the file should be removed
        reset_folder(result_folder)                                                                                     # Reset the folder
        print("Result folder reseted")                                                                                  # Print that the folder was reseted
    else:                                                                                                               # If the file shouldn't be removed
        print("Result folder already exists")                                                                           # Print that the folder already exists

## SETTING UP THE SIMULATIONS ------------------------------------------------------------------------------------------

num_cycles = 2                                                                                                          # Number of cycles in each simulation
simulations_num = 10                                                                                                    # Number of simulations to run
sampling_frequency = '30T'                                                                                              # Sampling frequency of the simulated data (30T = 30 minutes)
# Set seed
random.seed(42)

types = ['sine', 'square', 'sawtooth']                                                                                  # Types of simulated data
noise_levels = [0.1, 0.5, 1, 5, 10]

simulations = {}                                                                                                        # Dictionary to store the simulations data
for t in types:    
    simulations[t] = {}                                                                                                 # Create a dictionary to store the type data                                                                                       # Create a dictionary to store the noise level data
    for s in range(simulations_num):                                                                                    # For each simulation
        s = str(s)
        simulations[t][s] = {}                                                                                          # Create a dictionary to store the simulation data
        simulations[t][s]['cycles_days'] = [random.randint(3, 10) for _ in range(num_cycles)]                           # Randomly generate the number of days in each cycle (5 cycles)
        simulations[t][s]['activity_period'] = [random.randint(2200, 2600)/100 for _ in range(num_cycles)]              # Randomly generate the activity period in each cycle (5 cycles)

In [21]:
simulations

{'sine': {'0': {'cycles_days': [4, 3], 'activity_period': [25.79, 23.4]},
  '1': {'cycles_days': [6, 6], 'activity_period': [22.71, 25.77]},
  '2': {'cycles_days': [4, 4], 'activity_period': [25.02, 24.16]},
  '3': {'cycles_days': [3, 3], 'activity_period': [22.47, 23.11]},
  '4': {'cycles_days': [6, 3], 'activity_period': [24.87, 23.01]},
  '5': {'cycles_days': [9, 6], 'activity_period': [24.29, 25.01]},
  '6': {'cycles_days': [7, 3], 'activity_period': [25.88, 22.81]},
  '7': {'cycles_days': [9, 8], 'activity_period': [23.42, 22.79]},
  '8': {'cycles_days': [6, 8], 'activity_period': [22.52, 22.47]},
  '9': {'cycles_days': [9, 4], 'activity_period': [23.83, 23.76]}},
 'square': {'0': {'cycles_days': [7, 3], 'activity_period': [25.73, 24.35]},
  '1': {'cycles_days': [4, 9], 'activity_period': [22.4, 24.82]},
  '2': {'cycles_days': [7, 8], 'activity_period': [24.95, 22.98]},
  '3': {'cycles_days': [4, 3], 'activity_period': [25.38, 23.16]},
  '4': {'cycles_days': [7, 4], 'activity_peri

In [22]:
for t in types:    
    for s in simulations[t]:                                                                                            # For each simulation
        file_name = 'simulated_data_' + s + '_' + t                                                                     # Create the file name for the simulated data without noise
        cycle_days = simulations[t][s]['cycles_days']                                                                   # Get the number of days in each cycle
        activity_period = simulations[t][s]['activity_period']                                                          # Get the activity period in each cycle

        data_raw_without_noise = chs.simulate_protocol(file_name, save_folder, sampling_frequency, 
                                                       cycle_days, activity_period, signal_type = t, 
                                                       noise = False, snr_db = 20, only_positive = True, 
                                                       remove_file = True)                                              # Simulate the data with noise (chrono_simulation.py)
        simulations[t][s][file_name] = data_raw_without_noise                                                           # Store the data with noise in the simulations dictionary
        
        for n in noise_levels:       
            if len(str(n)) == 1:
                str_n = '0' + str(n) 
            else:
                str_n = str(n)
            file_name_noise = 'simulated_data_' + s + '_' + t + '_' + str_n                                             # Create the file name for the simulated data with noise
            data_raw_with_noise = chs.simulate_protocol(file_name_noise, save_folder, sampling_frequency, 
                                                        cycle_days, activity_period, signal_type = t, 
                                                        noise = True, snr_db = n, only_positive = True, 
                                                        remove_file = True)                                             # Simulate the data with noise (chrono_simulation.py)

        
            simulations[t][s][file_name_noise] = data_raw_with_noise                                                    # Store the data without noise in the simulations dictionary

        simulation_file = open(save_folder + '/simulation_' + s + '_' + t + '_description.txt', 'w')                    # Create a file to store the simulation description to be used later
        simulation_file.write('Simulation number: ' + s + '\n')                                                         # Write the simulation number
        simulation_file.write('Cycles days: ' + str(simulations[t][s]['cycles_days']) + '\n')                              # Write the cycles days
        simulation_file.write('Activity period: ' + str(simulations[t][s]['activity_period']) + '\n')                      # Write the activity period
        simulation_file.close()                                                                                         # Close the file

        print('Simulation ' + s + ' done!')                                                                             # Print a message to indicate that the simulation is done

Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy//simulated_data_0_sine.asc
Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy//simulated_data_0_sine_0.1.asc
Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy//simulated_data_0_sine_0.5.asc
Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy//simulated_data_0_sine_01.asc
Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy//simulated_data_0_sine_05.asc
Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy//simulated_data_0_sine_10.asc
Simulation 0 done!
Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy//simulated_data_1_sine.asc
Saved successfully in E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_c

In [8]:
fig = plt.figure(figsize = (12, 5))                                                                                     # Create a figure
ax = fig.add_subplot(111)                                                                                               # Add a subplot
ax.plot(data_raw_with_noise, color = 'maroon', linewidth = 2)                                                           # Plot the data with noise
ax.plot(data_raw_without_noise, color = 'dimgray', linewidth = 2)                                                       # Plot the data without noise
ax.set_xlabel('Time (Hours)')                                                                                           # Set the x label
ax.set_ylabel('Activity')                                                                                               # Set the y label
ax.legend(['With noise', 'Without noise'])                                                                              # Set the legend
plt.show()                                                                                                              # Show the plot

### IMPORT THE SIMULATIONS AND BUILD THE PROTOCOLS TO BE STUDIED

In [25]:
individual_files = glob.glob(save_folder + "\\*.asc")                                                                   # Get the files for each simulation (with and without noise) 

In [27]:
zt_0_time = 0                                                                                                           # Set the time for the first ZT (0)

simulations = {}                                                                                                        # Create a dictionary to store the simulations data
for file in individual_files:                                                                                           # For each file (each simulation)
    expression = r"\\simulated_data_(.*?)\_(.*?)\.asc"                                                                  # Create a regular expression to get the simulation number and if the data has noise or not
    name_file = re.search(expression, file)                                                                             # Search for the regular expression in the file name
    number = name_file.group(1)                                                                                         # Get the simulation number
    sig_noise = name_file.group(2)                                                                                      # Get if the data has noise or not
    
    if '_' in sig_noise:                                                                                                # If the data has noise
        sig = sig_noise.split('_')[0]                                                                                   # Get only the noise value
        noise = sig_noise.split('_')[1]                                                                                 # Get only the noise value
    else:                                                                                                               # If the data does not have noise
        sig = sig_noise                                                                                                 # Get the value
        noise = '0'                                                                                                     # Set the noise value to 0

    name = number + '_' + sig + '_' + noise                                                                             # Create the name of the simulation
    print(name)

    if sig not in simulations:                                                                                          # If the noise value is not in the dictionary
        simulations[sig] = {}
        simulations[sig][noise] = {}
    else:
        if noise not in simulations[sig]:
            simulations[sig][noise] = {}
    
    simulations[sig][noise][name] = {}                                                                                  # Create a dictionary to store each simulation data (each simulation)
    simulations[sig][noise][name]['file'] = name_file                                                                   # Store the file name
    simulation_file = save_folder + '/simulation_' + number + '_' + sig + '_description.txt'                            # Store the simulation description file name
    simulations[sig][noise][name]['simulation_file'] =simulation_file                                                   # Store the simulation description file name

    simulation_file = open(simulations[sig][noise][name]['simulation_file'], 'r')                                       # Open the simulation description file
    lines = simulation_file.readlines()                                                                                 # Read the lines
    simulation_file.close()                                                                                             # Close the file

    cycle_experssion = r"Cycles days: \[(.*?)\]"                                                                        # Create a regular expression to get the number of days in each cycle
    activity_expression = r"Activity period: \[(.*?)\]"                                                                 # Create a regular expression to get the activity period in each cycle
    cycle_days = re.search(cycle_experssion, lines[1]).group(1)                                                         # Get the number of days in each cycle
    activity_period = re.search(activity_expression, lines[2]).group(1)                                                 # Get the activity period in each cycle

    simulations[sig][noise][name]['cycle_days'] = [int(x) for x in cycle_days.split(', ')]                              # Store the number of days in each cycle
    simulations[sig][noise][name]['activity_period'] = [float(x) for x in activity_period.split(', ')]                  # Store the activity period in each cycle

    labels_dict = {'cycle_types': ['DD', 'DD'], 
                   'test_labels': ['1','2'], 
                   'cycle_days': simulations[sig][noise][name]['cycle_days']}                                           # Create a dictionary to store to pass as argument to the read_protocol function

    print(file)

    protocol = chr.read_protocol(name, file, zt_0_time = zt_0_time, labels_dict = labels_dict, 
                                 type = 'generic', consider_first_day = True)                                           # Read the protocol (chrono_reader.py)
    # protocol.resample('30T', method = 'sum')                                                                          # Resample the data (chrono_reader.py)
    # protocol.apply_filter(window = 3, type = 'moving_average', order = 2, reverse = False)                            # Apply a filter to the data (chrono_reader.py)
    # protocol.normalize_data(type = 'minmax', per_day = True)                                                          # Normalize the data (chrono_reader.py)

    simulations[sig][noise][name]['protocol'] = protocol                                                                # Store the protocol in the simulations dictionary

print(simulations)

0_sawtooth_0
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sawtooth.asc
0_sawtooth_0.1
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sawtooth_0.1.asc
0_sawtooth_0.5
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sawtooth_0.5.asc
0_sawtooth_01
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sawtooth_01.asc
0_sawtooth_05
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sawtooth_05.asc
0_sawtooth_10
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sawtooth_10.asc
0_sine_0
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sine.asc
0_sine_0.1
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/data_circadipy\simulated_data_0_sine_0.1.asc
0_sine_0.5
E:/github/nnc-ufmg/circ

### CONFIGURE THE FOLDERS TO RECEIVE THE ANALYSIS RESULTS

In [28]:
remove_file = False                                                                                                     # Set if the file should be removed or not if it already exists

print(result_folder)                                                                                                    # Print the results folder

def reset_folder(folder):                                                                                               # Function to reset the folder
    for root, dirs, files in os.walk(folder):                                                                           # For each file in the folder
        for f in files:                                                                                                 # For each file
            os.unlink(os.path.join(root, f))                                                                            # Remove the file
        for d in dirs:                                                                                                  # For each folder
            rmtree(os.path.join(root, d))                                                                               # Remove the folder

check_folder = os.path.isdir(result_folder)                                                                             # Check if the folder exists
if not check_folder:                                                                                                    # If the folder doesn't exist
    os.makedirs(result_folder)                                                                                          # Create the folder to save the file
    print("Results folder created")                                                                                     # Print that the folder was created
    for count, key in enumerate(list(simulations.keys())):                                                              # For each simulation
        for key_2 in list(simulations[key].keys()):                                                                     # For each key in the simulation   
            simulations[key][key_2]['save_folder'] = result_folder + '\\' + key + '_' + key_2                           # Set the folder to save the file
            check_simulation_folder = os.path.isdir(simulations[key][key_2]['save_folder'])                             # Check if the file exists in the folder
            if check_simulation_folder:                                                                                 # If the file exists
                if remove_file:                                                                                         # If remove_file is True
                    reset_folder(result_folder + '\\' + key + '_' + key_2)                                              # Reset the folder
                    print("Folder " + key + '_' + key_2 + " cleaned!")                                                  # Print that the folder was cleaned
                else:                                                                                                   # If remove_file is False
                    print("Folder already exists")                                                                      # Raise an error
            else:                                                                                                       # If the file doesn't exist
                os.makedirs(simulations[key][key_2]['save_folder'])                                                     # Create the folder to save the file
                print("Folder " + key + '_' + key_2 + " created!")                                                      # Print that the folder was created
else:                                                                                                                   # If the folder exists
    print("Results folder already exists")                                                                              # Print that the folder already exists
    for count, key in enumerate(list(simulations.keys())):                                                              # For each simulation
        for key_2 in list(simulations[key].keys()):                                                                     # For each key in the simulation    
            simulations[key][key_2]['save_folder'] = result_folder + '\\' + key + '_' + key_2                           # Set the folder to save the file
            check_simulation_folder = os.path.isdir(simulations[key][key_2]['save_folder'])                             # Check if the file exists in the folder
            if check_simulation_folder:                                                                                 # If the file exists
                if remove_file:                                                                                         # If remove_file is True
                    reset_folder(result_folder + '\\' + key + '_' + key_2)                                              # Reset the folder
                    print("Folder " + key + '_' + key_2 + " cleaned!")                                                  # Print that the folder was cleaned
                else:                                                                                                   # If remove_file is False
                    print("Folder already exists")                                                                      # Raise an error
            else:                                                                                                       # If the file doesn't exist
                os.makedirs(simulations[key][key_2]['save_folder'])                                                     # Create the folder to save the file
                print("Folder " + key + '_' + key_2 + " created!")                                                      # Print that the folder was created


E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/results_circadipy/
Results folder already exists
Folder sawtooth_0 created!
Folder sawtooth_0.1 created!
Folder sawtooth_0.5 created!
Folder sawtooth_01 created!
Folder sawtooth_05 created!
Folder sawtooth_10 created!
Folder sine_0 created!
Folder sine_0.1 created!
Folder sine_0.5 created!
Folder sine_01 created!
Folder sine_05 created!
Folder sine_10 created!
Folder square_0 created!
Folder square_0.1 created!
Folder square_0.5 created!
Folder square_01 created!
Folder square_05 created!
Folder square_10 created!


### RUN THE MODELING

- The "time_shape" can be 'continuous', 'meadian' or 'mean'. If it is 'continuous', all samples will be cosider to 
calculate the model. If it is 'median' or 'mean', the samples of activity will be grouped by mean/median along one day. 
- The "step" is the time step to calculate the model (in hours). 
- The "start/end_time" defines the period that the model will try to fit. E.g. if the start time is 24, the end time 
is 26, and the step is 0.5, the model will try to fit the data to a cosine curve with period equal to 24, 24.5, 25,
25.5 and 26 hours.
- The "n_components" is the number of components that the model will try to fit. If it is a list, the model will try to 
fit all the number of components in the list. E.g. if n_components = [1,2,3], the model will try to fit the data with
1, 2 and 3 components.

In [29]:
dict = {'time_shape': 'continuous',                     
        'step': 0.01, 
        'start_time': 22, 
        'end_time': 26, 
        'n_components': [1]}                                                                                                                            # Create a dictionary to pass as argument to the fit_cosinor function

best_models = []                                                                                                                                        # Create an empty list to store the best models

for count, sig in enumerate(simulations):                                                                                                               # For each simulation (simulation)
    for noise in simulations[sig]:                                                                                                                      # For each noise level
        result_folder = simulations[sig][noise]['save_folder']                                                                                          # Get the folder to save the file

        for sim in simulations[sig][noise]:                                                                                                             # For each simulation         
            if sim != 'save_folder':                                                                                          
                init = time.time()                                                                                                                      # Get the initial time

                best_models, _ = chrt.fit_cosinor(simulations[sig][noise][sim]['protocol'], dict = dict, save_folder = result_folder)                   # Fit the cosinor to the data (chrono_rithm.py)
                best_models_fixed, _ = chrt.fit_cosinor_fixed_period(simulations[sig][noise][sim]['protocol'], best_models, save_folder = result_folder)# Fix the best period calculated and fit the cosinor for each day using this period (chrono_rithm.py)
                simulations[sig][noise][sim]['best_models'] = best_models                                                                               # Store the best models in the simulations dictionary
                simulations[sig][noise][sim]['best_models_fixed'] = best_models_fixed                                                                   # Store the best models fixed in the simulations dictionary
                
                end = time.time() - init                                                                                                                # Get the time elapsed

                print("Cosinor fitted to " + sig + ' ' + noise + " and results saved!")                                                                 # Print that the cosinor was fitted and the results saved
                print("Time elapsed: " + "{:.2f}".format(end) + " seconds")                                                                             # Print the time elapsed

Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 15.72 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 10.68 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 17.62 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 16.36 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 16.22 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 10.81 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 13.67 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 12.38 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 12.54 seconds
Cosinor fitted to sawtooth 0 and results saved!
Time elapsed: 9.73 seconds
Cosinor fitted to sawtooth 0.1 and results saved!
Time elapsed: 11.76 seconds
Cosinor fitted to sawtooth 0.1 and results saved!
Time elapsed: 9.48 seconds
Cosinor fitted to sawtooth 0.1 and results saved!
Time elapsed: 13.57 seconds
Cosinor 

### COMPARE THE MODEL RESULTS FOR EACH CYCLE

In [30]:
read = True                                                                                                                               # If you already ran the analysis, set this to True to read the data instead of running the analysis again
model_error = {}                                                                                                                          # Create an empty list to store the model error
mean_model_error = {}                                                                                                                     # Create an empty list to store the mean model error
std_model_error = {}                                                                                                                      # Create an empty list to store the standard deviation of the model error
num_model_error = {}                                                                                                                      # Create an empty list to store the number of model error

for sig in simulations:
      model_error[sig] = {}                                                                                                               # Create an empty list to store the model error for each type of signal
      mean_model_error[sig] = {}                                                                                                          # Create an empty list to store the mean model error for each type of signal
      std_model_error[sig] = {}                                                                                                           # Create an empty list to store the standard deviation of the model error for each type of signal
      num_model_error[sig] = {}                                                                                                           # Create an empty list to store the number of model error for each type of signal
      for noise in simulations[sig]:
            model_error[sig][noise] = []                                                                                                  # Create an empty list to store the model error for each amplitude of noise            
            result_folder = simulations[sig][noise]['save_folder']                                                                        # Get the folder to save the file
            for sim in simulations[sig][noise]:                                                                                           # For each simulation (simulation)
                  if sim != 'save_folder':  
                        if read == True:
                              type_folder = sim.split('_', 1)[1]                                                                          # Get the type of signal
                              print(result_folder + '/cosinor_' + sim + '_.xlsx')
                              best_models = pandas.read_excel(result_folder + '/cosinor_' + sim + '_.xlsx')                               # Read the data
                              simulations[sig][noise][sim]['best_models'] = best_models                                                    # Store the data in the dictionary
                        
                        simulations[sig][noise][sim]['activity_period_model'] = list(simulations[sig][noise][sim]['best_models']['period']) # Get the activity period calculated by the model
                        
                        for stage in range(len(simulations[sig][noise][sim]['activity_period_model'])):
                              param = simulations[sig][noise][sim]['activity_period_model'][stage]
                              ground_truth = simulations[sig][noise][sim]['activity_period'][stage]
                              print(param, ground_truth)
                              simulation_model_error = param - ground_truth
                                                      
                              model_error[sig][noise].append(simulation_model_error)                                                                # Store the model error in the list

                        mean_model_error[sig][noise] = abs(numpy.mean(model_error[sig][noise]))                         # Calculate the mean model error
                        std_model_error[sig][noise] = numpy.std(model_error[sig][noise])                                # Calculate the standard deviation of the model error
                        num_model_error[sig][noise] = len(model_error[sig][noise])                                      # Calculate the number of model error

E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/results_circadipy/\sawtooth_0/cosinor_0_sawtooth_0_.xlsx
25.66000000000057 25.5
23.88000000000029 23.66
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/results_circadipy/\sawtooth_0/cosinor_1_sawtooth_0_.xlsx
22.0 22.16
23.34000000000021 23.61
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/results_circadipy/\sawtooth_0/cosinor_2_sawtooth_0_.xlsx
22.41000000000006 22.33
23.18000000000018 23.08
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/results_circadipy/\sawtooth_0/cosinor_3_sawtooth_0_.xlsx
25.51000000000055 25.35
24.60000000000041 24.55
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/results_circadipy/\sawtooth_0/cosinor_4_sawtooth_0_.xlsx
22.89000000000014 22.73
23.40000000000022 23.35
E:/github/nnc-ufmg/circadipy/src/analysis_examples/circadipy/results_circadipy/\sawtooth_0/cosinor_5_sawtooth_0_.xlsx
25.99000000000062 25.81
24.79000000000044 24.87
E:/github/nnc-ufmg

In [35]:
import pandas

model_error_df = pandas.DataFrame(model_error)
display(model_error_df)

mean_model_error_df = pandas.DataFrame(mean_model_error)
# Pass the first row to the last row
first_row = mean_model_error_df.iloc[0]
mean_model_error = pandas.concat([mean_model_error_df.iloc[1:], first_row.to_frame().T])
mean_model_error_df = mean_model_error_df.add_suffix('_mean')
display(mean_model_error_df)

std_model_error_df = pandas.DataFrame(std_model_error)
# Pass the first row to the last row
first_row = std_model_error_df.iloc[0]
std_model_error_df = pandas.concat([std_model_error_df.iloc[1:], first_row.to_frame().T])
std_model_error_df = std_model_error_df.add_suffix('_std')
display(std_model_error_df)

num_model_error_df = pandas.DataFrame(num_model_error)
# Pass the first row to the last row
first_row = num_model_error_df.iloc[0]
num_model_error_df = pandas.concat([num_model_error_df.iloc[1:], first_row.to_frame().T])
num_model_error_df = num_model_error_df.add_suffix('_num')
display(num_model_error_df)

unique_df = pandas.concat([mean_model_error_df, std_model_error_df, num_model_error_df], axis = 1)
display(unique_df)
# Save excel file
unique_df.to_excel(main_save_folder + '/ground_truth_comparison_results.xlsx')



,sawtooth,sine,square
0,"[0.16000000000056858, 0.22000000000029019, -0....","[5.897504706808832e-13, 2.2026824808563106e-13...","[0.06000000000058847, 0.3200000000004195, 0.04..."
0.1,"[0.49000000000062016, 0.7500000000003801, -0.1...","[-0.31999999999946027, 1.30000000000042, 0.110...","[0.26000000000061974, 0.2000000000003972, -0.0..."
0.5,"[0.08000000000055962, 0.13000000000027967, -0....","[-0.4999999999994884, 0.7900000000003402, 0.05...","[0.26000000000061974, -0.0999999999996497, 0.1..."
01,"[-0.7099999999995603, -0.3999999999997996, 0.1...","[0.20000000000062101, -0.4899999999998599, -0....","[-0.16999999999944038, 1.6400000000006187, -0...."
05,"[-0.1999999999994806, 0.4000000000003183, -0.1...","[-0.27999999999945047, 0.42000000000028237, 0....","[0.030000000000590887, 0.0500000000003773, -0...."
10,"[0.130000000000571, 0.48000000000033083, -0.16...","[0.14000000000061164, 0.1300000000002406, 0.05...","[0.11000000000059984, 0.5600000000004499, 0.02..."


,sawtooth_mean,sine_mean,square_mean
0.1,0.1445,8.550000e-02,0.0260
0.5,0.0075,1.070000e-01,0.0685
01,0.0085,3.900000e-02,0.0950
05,0.1065,5.150000e-02,0.0165
10,0.1050,1.200000e-02,0.0550
0,0.0665,2.900791e-13,0.0225


,sawtooth_std,sine_std,square_std
0.1,0.609815,4.721703e-01,0.275216
0.5,0.362269,4.270726e-01,0.232062
01,0.475755,2.338782e-01,0.453845
05,0.355321,2.221773e-01,0.142628
10,0.256310,8.947625e-02,0.142074
0,0.203452,1.797943e-13,0.087171


,sawtooth_num,sine_num,square_num
0.1,20,20,20
0.5,20,20,20
01,20,20,20
05,20,20,20
10,20,20,20
0,20,20,20


,sawtooth_mean,sine_mean,square_mean,sawtooth_std,sine_std,square_std,sawtooth_num,sine_num,square_num
0.1,0.1445,8.550000e-02,0.0260,0.609815,4.721703e-01,0.275216,20,20,20
0.5,0.0075,1.070000e-01,0.0685,0.362269,4.270726e-01,0.232062,20,20,20
01,0.0085,3.900000e-02,0.0950,0.475755,2.338782e-01,0.453845,20,20,20
05,0.1065,5.150000e-02,0.0165,0.355321,2.221773e-01,0.142628,20,20,20
10,0.1050,1.200000e-02,0.0550,0.256310,8.947625e-02,0.142074,20,20,20
0,0.0665,2.900791e-13,0.0225,0.203452,1.797943e-13,0.087171,20,20,20


In [41]:
## PLOT THE RESULTS OF THE COMPARISON ----------------------------------------------------------------------------------

# Plot the mean and std for each signal shape and each noise level
# Set the colors for each row
colors = ['gray', 'darkgray', 'lightsteelblue', 'cornflowerblue', 'royalblue', 'midnightblue']

# Create a figure and axis
fig, ax = plt.subplots()

# Set the width of each bar
bar_width = 0.15

# Create a NumPy array with the index values
x = numpy.arange(len(mean_model_error_df.columns))

# Plot each row as a separate bar with a specific color
for i, (index, row) in enumerate(mean_model_error_df.iterrows()):
    if index == '0':
        label_name = 'Without Noise'
    else:
        if index[0] == '0' and index[1] != '.':
            label_name = f'{index[1]}dB'
        else:
            label_name = f'{index}dB'
        
    # bar with error std
    ax.errorbar(x + (i * bar_width), row, yerr=std_model_error_df.loc[index], fmt='none', ecolor='black', capsize=3)
    ax.bar(x + (i * bar_width), row, bar_width, label=label_name, color=colors[i])

# Set the labels and title
ax.set_xticks(x + (2 * bar_width))
ax.set_xticklabels([c.upper().split('_', 1)[0] for c in mean_model_error_df.columns])

# Set legend to be in row
ax.legend(frameon=False, ncol=int(i/2))
ax.set_xlabel('SIGNAL SHAPE')
ax.set_ylabel('ERROR IN HOURS (CIRCADIPY - GROUND TRUTH) ')
ax.set_ylim(-0.5, 1)

# Show the plot
plt.tight_layout()
plt.show()

plt.savefig('comparison_circadipy.svg')


### PLOT THE RESULTS

It may take a while to run

In [48]:
format = 'svg'

for sig in simulations:
      for noise in simulations[sig]:
            for sim in simulations[sig][noise]:                                                                             # For each simulation (simulation)
                  result_folder = simulations[sig][noise]['save_folder']
                  if sim != 'save_folder': 
                        chp.actogram_bar(simulations[sig][noise][sim]['protocol'], first_hour = 18, save_folder = result_folder, save_suffix = 'bar', adjust_figure = [2, 0.95, 0.80, 0.20, 0.05], format = format)

                        chp.actogram_colormap(simulations[sig][noise][sim]['protocol'], first_hour = 18, save_folder = result_folder, save_suffix = 'colormap', adjust_figure = [2, 0.95, 0.80, 0.20, 0.05], norm_color = None, format = format)
                        chp.data_periodogram(simulations[sig][noise][sim]['protocol'], time_shape = 'continuous', method = 'periodogram', max_period = 48, unit_of_measurement = 'AMPLITUDE', save_folder = result_folder, save_suffix = 'periodogram', format = format)
                        chp.data_periodogram(simulations[sig][noise][sim]['protocol'], time_shape = 'continuous', method = 'welch', max_period = 48, unit_of_measurement = 'AMPLITUDE', save_folder = result_folder, save_suffix = 'welch', format = format)

                        chp.model_overview_detailed(simulations[sig][noise][sim]['protocol'], simulations[sig][noise][sim]['best_models_fixed'], save_folder = result_folder, format = format)
                        chp.model_overview(simulations[sig][noise][sim]['protocol'], simulations[sig][noise][sim]['best_models'], save_folder = result_folder, format = format)
                        chp.model_over_signal(simulations[sig][noise][sim]['protocol'], simulations[sig][noise][sim]['best_models'], position = 'head', mv_avg_window = 1, save_folder = result_folder, format = format)

                        chp.time_serie(simulations[sig][noise][sim]['protocol'], title = 'TIME SERIES', x_label = 'TIME (DAYS)', y_label = 'AMPLITUDE', save_folder = result_folder, format = format)

### RUN THE MODELING FOR SPECIFIED DAY WINDOWS

In [ ]:
dict = {'day_window': 1, 
        'step': 0.01, 
        'start_time': 22, 
        'end_time': 26, 
        'n_components': [1]}

best_models = []

for sig in simulations:
      for noise in simulations[sig]:
            for sim in simulations[sig][noise]:                                                                             # For each simulation (simulation)
                  result_folder = simulations[sig][noise]['save_folder']
                  if sim != 'save_folder': 
                        init = time.time()

                        best_models_per_day, best_models_per_day_file = chrt.fit_cosinor_per_day(simulations[sig][noise][sim]['protocol'], dict = dict, plot = True, save_folder = result_folder) 
                        chp.model_per_day(simulations[sig][noise][sim]['protocol'], best_models_per_day, dict['day_window'], save_folder = result_folder, save_suffix = '', format = format)   
                        simulations[sig][noise][sim]['best_models_per_day'] = best_models_per_day
                        simulations[sig][noise][sim]['best_models_file_per_day'] = best_models_per_day_file
